In [ ]:
#import libraries
from PIL import Image #work with images
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import zipfile
import os
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn

#path of the dataset zip file
path="/content/drive/MyDrive/Computer Vision Masterclass/Datasets/yalefaces.zip"

#uncompress the file
zip_obj = zipfile.ZipFile(file=path, mode='r')  #read mode
zip_obj.extractall('./')  #extract in current folder
zip_obj.close() #release memory

In [ ]:
#get np_representatin of image and class labels
def get_image_data(): #return id(class) and numpy representation of each image
  paths=[os.path.join('/content/yalefaces/train', f) for f in os.listdir('/content/yalefaces/train')] #path of train data
  #print(paths)
  faces=[]
  ids=[]
  for path in paths:
    #print(path)
    
    #get numpy representation of images
    image = Image.open(path).convert('L')   #L mode means gray scale mode (L- luminous)
    image_np = np.array(image, 'uint8')     #numpy representation of image
    faces.append(image_np)

    #get classes
    id = int(os.path.split(path)[1].split('.')[0].replace('subject',''))      #extracting class names by python string manipulations
    #print(id)
    ids.append(id)
  return np.array(ids), faces
    
ids, faces = get_image_data()

In [ ]:
#default parameters
#radius :1
#neighbors: 8
#grid_x: 8
#grid_y: 8
#threshold: 1.7976931348623157e+308

#training
lbph_classifier = cv2.face.LBPHFaceRecognizer_create(radius=4, neighbors=14, grid_x=9, grid_y=9)
lbph_classifier.train(faces, ids)
lbph_classifier.write('lbph_classifier.yml')    #save file- contains all histograms for each image

#load saved file
lbph_face_classifier = cv2.face.LBPHFaceRecognizer_create()
lbph_face_classifier.read('/content/lbph_classifier.yml') #loading the saved file

#evaluation
paths=[os.path.join('/content/yalefaces/test', f) for f in os.listdir('/content/yalefaces/test')]   #path of test data
predictions = []
expected_outs = []
for path in paths:
  image = Image.open(path).convert('L')   #L mode means gray scale mode (L- luminous)
  image_np = np.array(image, 'uint8')
  prediction, _ = lbph_face_classifier.predict(image_np)   #second return value is confidence level
  expected_out = int(os.path.split(path)[1].split('.')[0].replace('subject',''))  #extract class name

  #writing over the image about predicted and expected classes
  cv2.putText(image_np, 'Pred: '+str(prediction), (10,30), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0,255,0))
  cv2.putText(image_np, 'Exp: ' +str(expected_out),  (10,50), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0,255,0))
  cv2_imshow(image_np)  #only grayscale image possible

  predictions.append(prediction)
  expected_outs.append(expected_out)

In [ ]:
predictions = np.array(predictions)
expected_outs = np.array(expected_outs)
print('Accuracy :',accuracy_score(predictions, expected_outs)*100,'%',end='\n\n') #acuracy of predictions on test data

#confusion matrix
cm = confusion_matrix(predictions, expected_outs)

#heatmap visualisation
seaborn.heatmap(cm, annot=True);  #semicolon to hide message from matplotlib; annot - to see values